In [1]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes"
# folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
folders = ['webd-dependency', 'combined-fasttext', 'essays-dependency',
          'original', 'essays-fasttext', 'ibm-fasttext', 'webd-fasttext']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            if subdir.endswith('100'):
                print(path)
                indexref = pt.IndexRef.of(path)
                index = pt.IndexFactory.of(indexref)
                dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
                rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
                pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe

                param_map = {
                    rm3 : {
                        "fb_terms" : list(range(4, 16, 2)),
                        "fb_docs" : list(range(4, 10, 2)),
                        "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                    }
                }

                pipe_qe, parameters = pt.KFoldGridSearch(
                    pipe_qe,
                    param_map,
                    [topics20, topics21],
                    [qrels20, qrels21],
                    objective
                )
                print(pt.Experiment([pipe_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([pipe_qe], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm320'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([pipe_qe], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm321'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

                print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_dlm_rm3.csv")
                i += 1
            

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100/data.properties
Fold 1
Best ndcg_cut_5 is 0.334398
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7fe6ff5dacc0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55c7a6cfd9a0 at 0x7fe6ff797370>>) fb_terms=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7fe6ff5dacc0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55c7a6cfd9a0 at 0x7fe6ff797370>>) fb_docs=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7fe6ff5dacc0 jcla

/tmp/ipykernel_1663031/3362825214.py:39: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-dependency\_tuned\_dlm\_rm3 &    0.321377 &     0.233626 &  0.415613 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1663031/3362825214.py:40: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm320'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-dependency\_tuned\_dlm\_rm320 &     0.34853 &     0.253126 &  0.479037 \\
\bottomrule
\end{tabular}



...

\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_rm3 &    0.291709 &     0.217943 &  0.394852 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1663031/3362825214.py:40: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm320'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_rm320 &    0.273243 &     0.211706 &  0.441199 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1663031/3362825214.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm321'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_1663031/3362825214.py:43: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5

\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_rm321 &    0.309806 &     0.224056 &  0.349431 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_rm3\_unjudged &    0.535952 &     0.433532 &  0.394852 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                    name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_rm3\_unjudged20 &    0.662097 &     0.503915 &  0.441199 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                    name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_rm3\_unjudged21 &    0.555552 &     0.473513 &  0.349431 \\
\bottomrule
\end{tabular}



In [4]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print(path)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe

            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }

            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm320'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm321'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            i += 1
            

/workspace/src/grid_search_right_qrels/super_folder//data.properties
Fold 1
Best ndcg_cut_5 is 0.411759
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f8d755f8090 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x561f0b360950 at 0x7f8d7556ad10>>) fb_terms=4', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f8d755f8090 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x561f0b360950 at 0x7f8d7556ad10>>) fb_docs=6', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f8d755f8090 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x561f0b360950 at 0x7f8d7556ad10>>) fb_lambda=0.2']
Fold 2
Best ndcg_cut_5 is 0.258482
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f8

/tmp/ipykernel_2391493/4290711260.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391493/4290711260.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm320'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_rm3 &    0.321715 &      0.28292 &  0.625945 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_rm320 &     0.25162 &     0.246023 &  0.635986 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_rm321 &    0.390407 &     0.319079 &  0.616105 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_rm3\_unjudged &    0.630763 &      0.62729 &  0.625945 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\topru

/tmp/ipykernel_2391493/4290711260.py:38: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm321'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391493/4290711260.py:40: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5